# tilt


This notebook is background material for the Axial Seamount deformation signals data: Bottom pressure and tilt.



More web pages keep showing up from NOAA PMEL, such as https://www.pmel.noaa.gov/eoi/rsn/mj03d-2.html and https://www.pmel.noaa.gov/eoi/rsn/mj03d.html.


* `botpt` is pressure measured at about 2276 meters depth at decimal latitude 45.93363, longitude -130.01368
    * [OOI data team server](https://ooi-visualocean.whoi.edu/instruments/view/RS03ASHS-MJ03B-09-BOTPTA304)
    * [some data on an OpenDAP server](https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232309740Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/)
* `botpt` has an alterative at 15s sampling interval
* `botpt` has an alternative at 24hr sampling interval
* `iris` is the low resolution tilt meter
* `heat` is the coarse resolution tilt meter
* `lily` is the high resolution tilt meter
* `lily leveling` is presumed when the tilt meter resets


```
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample

RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232546825Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232840790Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232626646Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232354163Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s
```

## Needed


* Data retrieval story (not just 'I know a guy named Don')
* See this page: https://ooi-visualocean.whoi.edu/streams/view/botpt_lily_sample
* [Opendap](https://opendap.oceanobservatories.org/thredds/dodsC/ooi/ooicatest@gmail.com/20200810T232256217Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample_20200120T000001-20200810T224044.nc.html) allows us to streamline datasets...

From a [GitHub gist](https://gist.github.com/lsetiawan/14c1b92bd01f444f68ba5f496f11d237): 


```
import fsspec
import re
import pandas as pd
import requests
import os


import concurrent.futures

def map_concurrency(
    func, iterator, func_args=(), func_kwargs={}, max_workers=20
):
    """Run function on iterator with multiple threads"""
    results = []
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=max_workers
    ) as executor:
        # Start the load operations and mark each future with its URL
        future_to_url = {
            executor.submit(func, i, *func_args, **func_kwargs): i
            for i in iterator
        }
        for future in concurrent.futures.as_completed(future_to_url):
            data = future.result()
            results.append(data)
    return results
    
def url_download(url, data_fold):
    """ Method to help download files. """
    if not os.path.exists(data_fold):
        os.mkdir(data_fold)
    outpath = os.path.join(data_fold, os.path.basename(url))
    request = requests.get(url, stream=True)  # `prefetch=False` for older
    with open(outpath, "wb") as code:
        for i, chunk in enumerate(request.iter_content(100*(1024**2))):
            if chunk:
                code.write(chunk)
    if os.path.exists(outpath):
        return outpath

def extract_dates(url):
    """Extract dates from netcdf url"""
    req_ts, start_ts, end_ts = re.findall(r'\d{4}\d{2}\d{2}T\d{6}', url)
    return {
        'url': url,
        'req_ts': pd.to_datetime(req_ts),
        'start_ts': pd.to_datetime(start_ts),
        'end_ts': pd.to_datetime(end_ts)
    }

def filter_urls(nc_urls, name):
    """Filter URLS for OOINET Results"""
    filt_ds = list(
        filter(
            lambda x: (x.count(name) > 1)
            and (f"{name}_cal" not in x)
            and (f"{name}_blank" not in x),
            nc_urls,
        )
    )
    furls = list(filter(lambda nc: ".nc" in nc, filt_ds))
    return sorted([extract_dates(u) for u in furls], key=lambda i: i['start_ts'])

name = 'RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample'

url = 'https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232850498Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample'

htfs = fsspec.filesystem('http')

nc_urls = htfs.glob(os.path.join(url, '*.nc'))

nc_files = filter_urls(nc_urls, name)

out_folder = os.path.join('/data', name)

# Download only one of the file in this example. Extracting the url of the file from the dictionary.
nc_files = map_concurrency(url_download, [n['url'] for n in nc_files[:1]], func_args=(out_folder,))

nc_files
````






# prolix notes


## heat

Coarse resolution tilt meter. 


RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232256217Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample

```
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample_20170815T001037-20181107T235958.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample_20181108T000001-20200119T235958.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample_20200120T000001-20200810T224044.nc
```

## bottom pressure 'nano'
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232309740Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample


## iris
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232343358Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample


```
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20170815T001037-20180125T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20180126T000000-20180706T235959.nc 
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20180707T000000-20190102T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20190103T000001-20190613T235958.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20190614T000000-20191202T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20191203T000000-20200601T235959.nc 
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample_20200602T000000-20200810T224045.nc
```

## lily tilt
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample


About 500MB each:

```
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20170815T001037-20171226T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20171227T000000-20180507T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20180508T000000-20181006T235946.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20181007T000000-20190215T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190216T000000-20190701T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190702T000000-20191124T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20191125T000000-20200402T235959.nc
deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20200403T000000-20200810T224045.nc
```

## pressure 15s
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232354163Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s

One file for the entire time span (15 seconds implies a factor of 300 downsample)

deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20170815T001030-20200811T000000.nc

## pressure 24hr
RS03## heatASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_24hr : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232400202Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_24hr

Nothing there!

## lily leveling...
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_leveling : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232406021Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_leveling

## Unpacking nano (20 samples per second) bottom pressure



Data arrives from the OOI CI in NetCDF format. It contains noise and some structure around quality control. 



### Bottom pressure `botpt`


How many observations? 


```
len(ds.time)
```

results in `12724319`. This is (mostly) at 20 samples per second. 


What are the Data Variables?


```
ds.data_vars
```

gives


```
Data variables:
    preferred_timestamp          (obs) object ...
    sensor_id                    (obs) object ...
    bottom_pressure_qc_executed  (obs) uint8 ...
    date_time_string             (obs) object ...
    port_timestamp               (obs) datetime64[ns] ...
    press_trans_temp             (obs) float64 ...
    deployment                   (obs) int32 ...
    bottom_pressure_qc_results   (obs) uint8 ...
    id                           (obs) |S36 ...
    ingestion_timestamp          (obs) datetime64[ns] ...
    bottom_pressure              (obs) float32 ...
    internal_timestamp           (obs) datetime64[ns] ...
    time                         (obs) datetime64[ns] ...
    time_sync_flag               (obs) object ...
    driver_timestamp             (obs) datetime64[ns] ...
```

How many bytes per sample for these, in sum?


```
sum_bytes = 5 * np.dtype('datetime64').itemsize     # dt64 is 
sum_bytes += 1                                      # uint8 is one byte
sum_bytes += 4*8                                    # object is one byte
sum_bytes += 1*8                                    # float64 is 8 bytes
sum_bytes += 36                                     # |S36 is 36 bytes
sum_bytes += 4                                      # float32 is 4 bytes
sum_bytes += 4                                      # int32 is 4 bytes

print(len(ds.time)*sum_bytes, 'at bytes per obs = ', sum_bytes, 'not including obs overhead')
# how to get the size of some type in bytes: 

print(np.dtype('datetime64').itemsize)
print(np.dtype('uint8').itemsize)
print(np.dtype('object').itemsize)
print(np.dtype('float64').itemsize)
print(np.dtype('|S36').itemsize)
print(np.dtype('float32').itemsize)
print(np.dtype('int32').itemsize)
```

produces this output: 


```
1590539875 at bytes per obs =  125 not including obs overhead

8
1
8
8
36
4
4
```


#### How to examine the `botpt` XArray DataSet for content?


`dir(ds)` gives a lot of attributes; so here is a subset of these that are useful.


```
 'attrs',                                                      lists the attributes section

 'bottom_pressure'                                             the data
 'bottom_pressure_qc_executed'                                             ...what quality control?
 'bottom_pressure_qc_results'
 
 'coords',                                                     just 'obs'
 'data_vars'                                                   Data Variables. Note 'time' is present in addition to bottom_pressure (and other things)

 'dims'                                                        just 'obs'

 'geospatial_...                                               as expected

 'indexes'                                                     0, 1, ...
 'info'                                                        yet another way of seeing all these attributes
 'keys'                                                        and yet again: the dataset dictionary
 
 'lat'                                                           45.93363
 'lon'                                                         -130.01368

 'obs'                                                         obs

 'sensor'                                                     '09-BOTPTA304'

 'source'                                                     'RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample'

 'standard_name_vocabulary'                                   'NetCDF Climate and Forecast (CF) Metadata Convention Standard Name Table 29'

 'swap_dims'                                                  Useful to keep in mind for going to time rather than obs

 'time'                                                       treated as a data variable
 'time_coverage_end'                                          time range is Aug 15 - 22 2017; 7 days 13 hrs > consistent with 13 millions samples
 'time_coverage_resolution'                                   .05 seconds
 'time_coverage_start'

 'title'
 'to_<other_format>'                                           re-casting, includes zarr as an option

 'values'                                                      useful, same as 'var'
 'var'                                                         useful, same as 'values'
 'variables'                                                   unreadable
```


```
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232256217Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_heat_sample
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232309740Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232343358Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_iris_sample
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232354163Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_24hr : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232400202Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_24hr
RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_leveling : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232406021Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_leveling


RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_status : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232417546Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_status
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_24hr : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232420031Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_24hr
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232431313Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_leveling : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232543927Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_leveling
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_heat_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232544474Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_heat_sample
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232546825Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample
RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_iris_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232608319Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_iris_sample


RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_iris_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232617985Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_iris_sample
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_leveling : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232626185Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_leveling
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232626646Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232633263Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_heat_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232756784Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_heat_sample
RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_24hr : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232759333Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_24hr


RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_status : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232814379Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_status
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_iris_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232814543Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_iris_sample
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_leveling : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232824138Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_leveling
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_heat_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232824860Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_heat_sample
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_24hr : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232828520Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_24hr
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232840790Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s
RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample : https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232850498Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample
```



In [2]:
# placeholder for initialization code (prints Python version, imports, etcetera)

# lily

In [1]:
# In Python we use !wget with -P to re-point the output to the data directory. Here are the data files as of 25-SEP-2020.
# 
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20170815T001037-20171226T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20171227T000000-20180507T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20180508T000000-20181006T235946.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20181007T000000-20190215T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190216T000000-20190701T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190702T000000-20191124T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20191125T000000-20200402T235959.nc
# deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20200403T000000-20200810T224045.nc

# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20171227T000000-20180507T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20180508T000000-20181006T235946.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20181007T000000-20190215T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190216T000000-20190701T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20190702T000000-20191124T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20191125T000000-20200402T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232348347Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_lily_sample_20200403T000000-20200810T224045.nc

# pressure 15 sec

In [2]:
# !wget https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232354163Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0003_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20170815T001030-20200811T000000.nc
# was 156MB

In [3]:
# 301 302 303 (as 304 = ASHES already done)


tilt_urls = ['https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample',\
             'https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample',\
             'https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample']

pressure_urls = ['https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232546825Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s',\
                 'https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232840790Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s',\
                 'https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232626646Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s']


In [6]:
tilt_names = ['RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample',\
              'RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample',\
              'RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample']

pressure_names = ['RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s',\
                  'RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s',\
                  'RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s']


In [7]:
import fsspec
import re
import pandas as pd
import requests
import os

import concurrent.futures

In [8]:
def map_concurrency(func, iterator, func_args=(), func_kwargs={}, max_workers=20):
    """Run function on iterator with multiple threads"""
    results = []
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=max_workers
    ) as executor:
        # Start the load operations and mark each future with its URL
        future_to_url = {
            executor.submit(func, i, *func_args, **func_kwargs): i
            for i in iterator
        }
        for future in concurrent.futures.as_completed(future_to_url):
            data = future.result()
            results.append(data)
    return results
    
def url_download(url, data_fold):
    """ Method to help download files. """
    if not os.path.exists(data_fold):
        os.mkdir(data_fold)
    outpath = os.path.join(data_fold, os.path.basename(url))
    request = requests.get(url, stream=True)  # `prefetch=False` for older
    with open(outpath, "wb") as code:
        for i, chunk in enumerate(request.iter_content(100*(1024**2))):
            if chunk:
                code.write(chunk)
    if os.path.exists(outpath):
        return outpath

def extract_dates(url):
    """Extract dates from netcdf url"""
    req_ts, start_ts, end_ts = re.findall(r'\d{4}\d{2}\d{2}T\d{6}', url)
    return {
        'url': url,
        'req_ts': pd.to_datetime(req_ts),
        'start_ts': pd.to_datetime(start_ts),
        'end_ts': pd.to_datetime(end_ts)
    }

def filter_urls(nc_urls, name):
    """Filter URLS for OOINET Results"""
    filt_ds = list(
        filter(
            lambda x: (x.count(name) > 1)
            and (f"{name}_cal" not in x)
            and (f"{name}_blank" not in x),
            nc_urls,
        )
    )
    furls = list(filter(lambda nc: ".nc" in nc, filt_ds))
    return sorted([extract_dates(u) for u in furls], key=lambda i: i['start_ts'])

In [9]:
name = tilt_names[0]
url = tilt_urls[0]
htfs = fsspec.filesystem('http')
nc_urls = htfs.glob(os.path.join(url, '*.nc'))
nc_files = filter_urls(nc_urls, name)
out_folder = os.path.join('/data/botpt', name)

In [10]:
# Download only one of the file in this example. Extracting the url of the file from the dictionary.
nc_files = map_concurrency(url_download, [n['url'] for n in nc_files[:1]], func_args=(out_folder,))
nc_files

['/data/botpt/RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20140829T205940-20150129T235959.nc']

# tilt 301 wget

In [13]:
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20140829T205940-20150129T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20150130T000000-20150626T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20150627T000000-20151115T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20151116T000000-20160324T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20160325T000000-20160803T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20160804T000000-20161211T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20161212T000000-20170420T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20170421T000000-20170828T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20170829T000000-20180107T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20180108T000000-20180519T235959.nc 
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20180520T000000-20181020T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20181021T000000-20190223T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20190224T000000-20190703T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20190704T000000-20191118T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20191119T000000-20200325T235959.nc
# !wget -P /data1/botpt/lily/ https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232557746Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_lily_sample_20200326T000000-20200810T224045.nc

# tilt 302 wget

In [15]:
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20140903T215025-20150123T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20150124T000000-20150620T235959.nc 
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20150621T000000-20151107T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20151108T000000-20160316T235959.nc 
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20160317T000000-20160728T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20160729T000000-20161207T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20161208T000000-20170418T235959.nc 
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20170419T000000-20170830T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20170831T000000-20180113T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20180114T000000-20180527T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20180528T000000-20181022T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20181023T000000-20190225T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20190226T000000-20190705T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20190706T000000-20191116T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20191117T000000-20200325T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232806351Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_lily_sample_20200326T000000-20200810T224045.nc

# tilt 303 wget

In [17]:
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20140829T231723-20150121T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20150122T000000-20150618T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20150619T000000-20151111T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20151112T000000-20160322T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20160323T000000-20160803T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20160804T000000-20161213T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20161214T000000-20170422T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20170423T000000-20170830T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20170831T000000-20180113T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20180114T000000-20180525T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20180526T000000-20181026T235959.nc 
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20181027T000000-20190327T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20190328T000000-20190808T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20190809T000000-20191220T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20191221T000000-20200426T235959.nc
# !wget -P /data1/botpt/lily https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232748565Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_lily_sample_20200427T000000-20200810T224045.nc

# bottom pressure 301 302 303 wget

In [1]:
# !wget -P /data1/botpt/pressure https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232546825Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s_20140829T205945-20200811T000000.nc
# !wget -P /data1/botpt/pressure https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232840790Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s_20140903T220515-20200811T000000.nc
# !wget -P /data1/botpt/pressure https://opendap.oceanobservatories.org/async_results/ooicatest@gmail.com/20200810T232626646Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s_20140829T231730-20200811T000000.nc

# testing bottom_pressure 15-second files... why so slow?

In [4]:
pressurefiles = glob.glob('/data/botpt/pressure/deployment*.nc')
pressurefiles

['/data/botpt/pressure/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s_20140829T205945-20200811T000000.nc',
 '/data/botpt/pressure/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s_20140903T220515-20200811T000000.nc',
 '/data/botpt/pressure/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_15s_20140829T231730-20200811T000000.nc']